# Experiment Notebook 
#### This notebook is meant to experiment with different parameters, architectures, and normalization methods associated with the gRNA2knockdown Library.
#### The basic characteristic of the model as developed by Enoch Yeung are still preserved in the __main__ function of the module.

#### First Let's import or data, the description of which is below:
- The sequencing data is imported as a CSV and contains gRNA designs which tile the target (GFP) with 3bp of overlap.
- The time series data is imported as a TSV text file generated BioTek's Gen5 software and from data collected on Synergy H1s.
- The experimental design is:
  - Total run time = 18 hours
  - Time step = 3 min.
  - Reads:
    - OD600: Absorbance at 600 nm
    - Fluorescence: Excitation = 480 nm, Emission = 510 nm.
  - Runs where conducted with breath-easy film to reduce moisture loss.

In [ ]:
# First we need to load the data
data_fp = "Data/"
spacer_fp = os.path.join(data_fp, "GFP_spacers.gbk")
data_0nM_fp = os.path.join(data_fp,
                            "p2x11_80memberlibrary_0mMIPTG20230730.txt")
data_10mM_fp = os.path.join(data_fp,
                            "p2x11_80memberlib_10mMIPTG20230730.txt")

# Organize the label, sequence data from platereadertools and the csv standard module.
seqs = csv.reader(open("Data/GFP_spacers.csv"))
allseqs = [seq for seq in seqs]
data0, time0 = prt.Organize(data_0nM_fp, 8, 12, 18, 3/60)
data1, time1 = prt.Organize(data_10mM_fp, 8, 12, 18, 3/60)

# Based off of the timeseries data, we can see that the greatest change in flourescence occurs at timepoint 165 
# (~8hours). We will use this timepoint to calculate the fold change between the 0mM and 10mM data.
reads = list(data0.keys())
data_pt0 = data0[reads[1]][:,:,165]
data_pt1 = data1[reads[1]][:,:,165]

# Calculate the fold change between the 0mM and 10mM data.
fold_change = data_pt1/data_pt0
data = np.reshape(fold_change,(96))

: 

In [ ]:
fig, ax = plt.subplots(1,1)
ax.bar(range(len(data)), data, color = 'yellow')
ax.set_xlabel("gRNA position with tilling equal to 3bp")
ax.set_ylabel('Fold Change in RFU')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.set_title("Fold Change in RFU for each gRNA position at 8 hours")
if True:
    plt.savefig("./Figures/foldchange.png")

: 

### Define the gRNA2knockdown model parameters.
- The size of the gRNAs is 30 bp, so it makes may make sense to make the intermediate layers 30 nodes.
- For now lets keep the embedding space the same size, and not try to use the resnet functionality.
- Get a slot for the rack computer (email Josh).

In [ ]:
stride_parameter = 25 #
label_dim = 1
embedding_dim = 24
intermediate_dim = 40 # Make this larger than the sequencing space
batch_size_parameter=5 # Should be the same as the number of gRNAs
debug_splash = 0
this_step_size_val = 0.1
this_corpus, this_labels = g2k.make_labeled_corpus(allseqs, data,
                                                    stride_parameter)
sess = None

: 

In [ ]:
# Define the random transformation householder matrix.
Rand_Transform = g2k.rvs(dim=stride_parameter)

# Define the corpus for the model.
this_corpus_vec = []
for this_corpus_elem in this_corpus:
    vec_value = g2k.sequence_encoding(this_corpus_elem)
    vec_value = np.dot(Rand_Transform,vec_value)
    this_corpus_vec.append(vec_value)

this_corpus_vec = np.asarray(this_corpus_vec)
this_labels = np.expand_dims(this_labels,axis=1)
hidden_vars_list = [stride_parameter, embedding_dim, stride_parameter]
print(hidden_vars_list)

: 

In [ ]:
# Data is mostly redundant, there are sparse features to extract from redundant data. 
this_corpus


: 

### Build the model.

In [ ]:
# Define the tensorflow session
if sess != None:
        sess.close()
sess = tf.compat.v1.Session()
tf.compat.v1.disable_eager_execution() # needed because of placeholder variables

this_u = tf.compat.v1.placeholder(tf.float32, 
                                    shape=[None,stride_parameter])

with tf.device('/cpu:0'):
    this_W_list,this_b_list = g2k.initialize_Wblist(stride_parameter,
                                                hidden_vars_list)
    this_y_out,all_layers = g2k.network_assemble(this_u,this_W_list,this_b_list
                                            ,keep_prob=1.0,
                                            activation_flag=2,res_net=0)

this_embedding = all_layers[-2]
regress_list = [intermediate_dim]*7+[label_dim]
with tf.device('/cpu:0'):
    this_Wregress_list,this_bregress_list = g2k.initialize_Wblist(embedding_dim,regress_list)


: 

In [ ]:

HybridLoss = g2k.customLoss(this_y_out,this_u,this_embedding)
this_optim = tf.compat.v1.train.AdagradOptimizer(
    learning_rate=this_step_size_val).minimize(g2k.vae_loss(this_y_out,this_u))
step_size = tf.compat.v1.placeholder(tf.float32,shape=[])
result = sess.run(tf.compat.v1.global_variables_initializer())
this_vae_loss = g2k.vae_loss(this_y_out,this_u)
this_embed_loss = g2k.embed_loss(this_u,this_embedding)

: 

In [ ]:
g2k.train_net(sess, this_corpus_vec, 
              this_u, HybridLoss,
              this_optim,
              batchsize=batch_size_parameter, 
              this_vae_loss=this_vae_loss, 
              this_embed_loss=this_embed_loss,
              step_size_val=this_step_size_val*0.1, 
              valid_error_thres=1e-4,
              test_error_thres=1e-4, max_iters=10e4,
              save_fig='Figures/stepsize0_05.png'
              )

: 

In [ ]:
print(this_y_out.eval(feed_dict={this_u:[this_corpus_vec[2]]}, session=sess))
z_ind = this_y_out.eval(feed_dict={this_u:[this_corpus_vec[2]]}, session=sess)
print(this_corpus_vec)
print(this_corpus)

: 

In [ ]:
import seaborn as sns;
all_mismatches = []
for ind in range(0,len(this_corpus_vec)):
    z_ind = this_y_out.eval(feed_dict={this_u:[this_corpus_vec[ind]]}, session=sess)
    this_seq_out = g2k.vecback2seq(np.dot(np.linalg.inv(Rand_Transform),z_ind.T))
    print(g2k.vecback2seq(np.dot(np.linalg.inv(Rand_Transform),z_ind.T)))
    this_seq_out = ''.join(this_seq_out)
    all_mismatches.append(g2k.num_mismatch(this_seq_out,this_corpus[ind]));
hist_data = sns.displot(all_mismatches)
mismatch_process = np.array(all_mismatches);
mismatch_process[mismatch_process<=2.0] = 1.0;
mismatch_process[mismatch_process>2.0] = 0.0;
np.sum(mismatch_process)/(len(mismatch_process)*1.0)

: 

# Hyperparameter Sweeping
Try to sweep various hyperparameters to find an optimal model parameter.